# Geração de scripts de TV
Neste projeto, você gerará seus próprios scripts de TV [Simpsons] (https://en.wikipedia.org/wiki/The_Simpsons) usando RNNs. Você estará usando parte do [conjunto de dados Simpsons] (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) de scripts de 27 temporadas. A Rede Neural que você irá construir irá gerar um novo roteiro de TV para uma cena em [Moe's Tavern] (https://simpsonswiki.com/wiki/Moe's_Tavern).
## Obtém os dados
Os dados já são fornecidos para você. Você estará usando um subconjunto do conjunto de dados original. Consiste apenas nas cenas da Taverna de Moe. Isso não inclui outras versões da taverna, como "Moe's Cavern", "Flame Moe's", "Uncle Moe's Family Feed-Bag", etc.

In [1]:
import tensorflow as tf
tf.__version__

'1.1.0'

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
!cat requirements.txt | grep numpy 

numpy==1.12.1


In [9]:
!pip3 install --upgrade numpy

  Using cached https://files.pythonhosted.org/packages/f5/bf/4981bcbee43934f0adb8f764a1e70ab0ee5a448f6505bd04a87a2fda2a8b/numpy-1.16.1-cp36-cp36m-manylinux1_x86_64.whl


In [14]:
#!pip3 uninstall numpy

In [4]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implementar funções de pré-processamento
A primeira coisa a fazer em qualquer conjunto de dados é o pré-processamento. Implemente as seguintes funções de pré-processamento abaixo:
- Tabela de pesquisa
- Tokenize pontuação

### Tabela de pesquisa
Para criar uma incorporação de palavras, você primeiro precisa transformar as palavras em ids. Nesta função, crie dois dicionários:
- Dicionário para ir das palavras para um id, vamos chamar `vocab_to_int`
- Dicionário para ir do id para a palavra, vamos chamar `int_to_vocab`

Devolve estes dicionários na seguinte tupla `(vocab_to_int, int_to_vocab)`

In [5]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ix for ix, word in enumerate(vocab)}
    int_to_vocab = {ix: word for word, ix in vocab_to_int.items()}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
Estaremos dividindo o script em um array de palavras usando espaços como delimitadores. No entanto, pontuações como períodos e pontos de exclamação tornam difícil para a rede neural distinguir entre a palavra "bye" e "bye!".

Implemente a função `token_lookup` para retornar um comando que será usado para tokenizar símbolos como"! " em "|| Exclamation_Mark ||". Crie um dicionário para os seguintes símbolos, onde o símbolo é a chave e o valor é o token:
- Período (.)
- vírgula (,)
- Aspa ( " )
- ponto e vírgula (;)
- Ponto de exclamação ( ! )
- Ponto de interrogação ( ? )
- Parêntese esquerdo (()
- Parênteses direitos ())
- traço (-)
- retorno (\ n)

Este dicionário será usado para simbolizar os símbolos e adicionar o delimitador (espaço) ao redor dele. Isso separa os símbolos como sua própria palavra, facilitando a previsão da rede neural na próxima palavra. Certifique-se de não usar um token que possa ser confundido como uma palavra. Em vez de usar o token "dash", tente usar algo como "|| dash ||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # Implement Function
    token_dict = {}
    token_dict['.'] = "||Period||"
    token_dict[','] = "||Comma||"
    token_dict['"'] = "||Quotation_Mark||"
    token_dict[';'] = "||Semicolon||"
    token_dict['!'] = "||Exclamation_Mark||"
    token_dict['?'] = "||Question_Mark||"
    token_dict['('] = "||Left_Parentheses||"
    token_dict[')'] = "||Right_Parentheses||"
    token_dict['--'] = "||Dash||"
    token_dict['\n'] = "||Return||"
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pré-processe todos os dados e salve-os
A execução da célula de código abaixo irá pré-processar todos os dados e salvá-los no arquivo.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
Este é seu primeiro checkpoint. Se você decidir voltar a este notebook ou precisar reiniciar o notebook, poderá começar a partir daqui. Os dados pré-processados ​​foram salvos no disco.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Construa a Rede Neural
Você construirá os componentes necessários para construir um RNN implementando as seguintes funções abaixo:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Verifique a versão do TensorFlow e acesse o GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/joana/.local/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Entrada
Implementar o `
get_inputs () `para criar espaços reservados para TF para a rede neural. Deve criar os seguintes marcadores de posição:
- Espaço reservado para entrada de texto chamado "input" usando o parâmetro [TF Placeholder] (https://www.tensorflow.org/api_docs/python/tf/placeholder) `name`.
- Espaço reservado para alvos
- Espaço reservado para Learning Rate

Retorna os placeholders na seguinte tupla `(Input, Targets, LearningRate)`

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # Implement Function
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Construir Célula RNN e Inicializar
Empilhe uma ou mais [`BasicLSTMCells`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) em um [` MultiRNNCell`] (https://www.tensorflow.org/ api_docs / python / tf / contrib / rnn / MultiRNNCell).
- O tamanho do Rnn deve ser definido usando `rnn_size`
- Initalize Cell State usando a função [`zero_state ()`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) do MultiRNNCell
    - Aplique o nome "initial_state" ao estado inicial usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna a célula e o estado inicial na seguinte tupla `(Cell, InitialState)`

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # TODO: no dropout?
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * 1) # TODO: parameter rnn_layer_size?
    zero_state = cell.zero_state(batch_size, tf.float32) # TODO: int32 or float32?
    initial_state = tf.identity(zero_state, name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Incorporação de palavras
Aplique a incorporação ao `input_data` usando o TensorFlow. Retornar a seqüência incorporada.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # Implement Function
    embedding = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Construir RNN
Você criou uma célula RNN na função `get_init_cell ()`. Hora de usar a célula para criar um RNN.
- Construa o RNN usando o [`tf.nn.dynamic_rnn ()`] (https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Aplique o nome "final_state" ao estado final usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna as saídas e o estado final_state na seguinte tupla `(Saídas, FinalState)`

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # Implement Function
    output, fin_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32) # TODO: float32 or int32
    # TODO: Pass in initial_state?
    final_state = tf.identity(fin_state, name="final_state")
    return output, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Construa a Rede Neural
Aplique as funções que você implementou acima para:
- Aplicar incorporação ao `input_data` usando sua função` get_embed (input_data, vocab_size, embed_dim) `.
- Construa RNN usando `cell` e sua função` build_rnn (cell, inputs) `.
- Aplique uma camada conectada _com uma ativação _linear_ e `vocab_size` como o número de saídas.

Retorna os logits e o estado final na seguinte tupla (Logits, FinalState)

In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    output, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(output, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Lotes
Implemente `get_batches` para criar lotes de entrada e de destino usando` int_text`. Os lotes devem ser um array Numpy com a forma `(número de lotes, 2, tamanho do lote, comprimento da seqüência)`. Cada lote contém dois elementos:
- O primeiro elemento é um único lote de ** entrada ** com a forma `[tamanho do lote, comprimento da sequência]`
- O segundo elemento é um único lote de ** destinos ** com a forma `[tamanho do lote, comprimento da sequência]`

Se você não puder preencher o último lote com dados suficientes, elimine o último lote.

Por exemplo, `get_batches ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) `retornaria uma matriz Numpy dos seguintes:
`` `
[
  # Primeiro lote
  [
    # Lote de entrada
    [[1 2], [7 8], [13 14]]
    # Lote de alvos
    [[2 3], [8 9], [14 15]]
  ]

  # Segundo lote
  [
    # Lote de entrada
    [[3 4], [9 10], [15 16]]
    # Lote de alvos
    [[4 5], [10 11], [16 17]]
  ]

  # Terceiro lote
  [
    # Lote de entrada
    [[5 6], [11 12], [17 18]]
    # Lote de alvos
    [[6 7], [12 13], [18 1]]
  ]
]
`` `

Observe que o último valor de destino no último lote é o primeiro valor de entrada do primeiro lote. Neste caso, `1`. Essa é uma técnica comum usada na criação de lotes de sequências, embora não seja intuitiva.

In [15]:
def get_batches(int_text, batch_size, seq_length):
    # Implement Function
    chunk_size = batch_size * seq_length
    num_chunks = len(int_text) // chunk_size
    
    # take care of the wraparound
    if (num_chunks*chunk_size==len(int_text)):
        int_text = int_text + int_text[0:1]
    else:
        int_text[num_chunks*chunk_size] = int_text[0]
    
    # sequence starting indices
    si = [i for i in range(0, len(int_text)-seq_length+1, seq_length)]

    batches = []
    for i in range(num_chunks):
        x = [] # input batch
        y = [] # target batch
        for j in range(batch_size):
            # pick batch_size sequences starting at start and spaced every num_chunks
            start = si[j*num_chunks + i]
            x.append(int_text[start:start+seq_length])
            # advance for corresponding target
            start += 1
            y.append(int_text[start:start+seq_length])
        batches.append([x, y])
    
    return np.array(batches)

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [17]:
# Number of Epochs
num_epochs = 35
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.005
# Show stats for every n number of batches
show_every_n_batches = 25

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/44   train_loss = 8.821
Epoch   0 Batch   25/44   train_loss = 5.960
Epoch   1 Batch    6/44   train_loss = 5.248
Epoch   1 Batch   31/44   train_loss = 5.195
Epoch   2 Batch   12/44   train_loss = 4.765
Epoch   2 Batch   37/44   train_loss = 4.700
Epoch   3 Batch   18/44   train_loss = 4.491
Epoch   3 Batch   43/44   train_loss = 4.407
Epoch   4 Batch   24/44   train_loss = 4.124
Epoch   5 Batch    5/44   train_loss = 3.928
Epoch   5 Batch   30/44   train_loss = 3.948
Epoch   6 Batch   11/44   train_loss = 3.789
Epoch   6 Batch   36/44   train_loss = 3.739
Epoch   7 Batch   17/44   train_loss = 3.471
Epoch   7 Batch   42/44   train_loss = 3.364
Epoch   8 Batch   23/44   train_loss = 3.255
Epoch   9 Batch    4/44   train_loss = 3.121
Epoch   9 Batch   29/44   train_loss = 3.056
Epoch  10 Batch   10/44   train_loss = 3.005
Epoch  10 Batch   35/44   train_loss = 2.879
Epoch  11 Batch   16/44   train_loss = 2.821
Epoch  11 Batch   41/44   train_loss = 2.714
Epoch  12 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [22]:
tf.__version__

'1.1.0'

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [23]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Escolher palavra
Implemente a função `pick_word ()` para selecionar a próxima palavra usando `probabilities`.

In [24]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    idx = np.argmax(probabilities)
    return int_to_vocab[idx]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)



Tests Passed


## Gerar script de TV
Isso irá gerar o script de TV para você. Defina `gen_length` com o comprimento do script de TV que você deseja gerar.

In [29]:
gen_length = 300
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson:(reluctant) all right, i need both hands for this.
moe_szyslak:(chuckles) and now, i'd say you a little bit.
moe_szyslak:(to self) someone's lookin' at a snake in this bar.
homer_simpson:(chuckles) jokes on, it works.
homer_simpson:(chuckles) jokes on, it was flanders. now, you gotta stay out on the time.
moe_szyslak: so, uh, you were done here, homer.(nervous chuckle)
moe_szyslak: you little puke...
moe_szyslak:(cutting him off) oh, right. and you accept the charges?
c. _montgomery_burns: but then, lisa, it's not like sloppy, it's the good life.
homer_simpson:(chuckles) jokes on, it works.
homer_simpson:(chuckles) jokes on, it was flanders. now, you gotta stay out on the time.
moe_szyslak: so, uh, you were done here, homer.(nervous chuckle)
moe_szyslak: you little puke...
moe_szyslak:(cutting him off) oh, right. and you accept the charges?
c. _montgomery_burns: but then, lisa, it's not like sloppy, it's the good life.
home

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.